**Trying to approximate Lorentz using SINDy**

Note before reading the code: This code is experimental, I tried using small number of data with different approaches to approximate the Lorentz equations. If I used high number of samples the code would've converged on cell 2 and no further experimentation is needed.

Cell 1, construction of the feature space


In [150]:
## Cell 1
import pandas as pd
import numpy as np
import itertools
import os

# File path
file_path = r"C:\Users\braid\OneDrive\Desktop\lorenze_attractor.csv\lorenze_attractor.csv"

# Load Data
df = pd.read_csv(file_path)
df = df.drop(df.columns[0], axis=1)  # drop sample index

expected_cols = ["x", "y", "z", "u", "t"]
for c in expected_cols:
    if c not in df.columns:
        raise ValueError(f"Missing column '{c}' in CSV. Found: {df.columns.tolist()}")

x, y, z, u, t = [df[c].to_numpy() for c in expected_cols]
dt = np.mean(np.diff(t))

# Using Finite Difference for derivatives (FDM)
def finite_diff(signal, dt):
    deriv = np.zeros_like(signal)
    deriv[1:-1] = (signal[2:] - signal[:-2]) / (2 * dt)
    deriv[0] = (signal[1] - signal[0]) / dt
    deriv[-1] = (signal[-1] - signal[-2]) / dt
    return deriv

dx, dy, dz, du = [finite_diff(var, dt) for var in [x, y, z, u]]

# Building the feature space
features = {
    "x": x, "y": y, "z": z, "u": u, "t": t,
    "dx": dx, "dy": dy, "dz": dz, "du": du
}

# Polynomial powers
for var in ["x", "y", "z", "u"]:
    features[f"{var}^2"] = df[var] ** 2
    features[f"{var}^3"] = df[var] ** 3

# Pairwise products
vars_basic = ["x", "y", "z", "u"]
for i, j in itertools.combinations_with_replacement(vars_basic, 2):
    features[f"{i}{j}"] = df[i] * df[j]

# Derivative interactions
for var in vars_basic:
    for dvar in ["dx", "dy", "dz", "du"]:
        features[f"{var}*{dvar}"] = df[var] * features[dvar]

# Cross derivative terms
for i, j in itertools.combinations_with_replacement(["dx", "dy", "dz", "du"], 2):
    features[f"{i}*{j}"] = features[i] * features[j]

# Complex higher-order terms
complex_terms = [
    ("x", "dx", "dy"), ("x", "dy", "dz"), ("y", "dx", "dz"),
    ("z", "dx", "dy"), ("x", "y", "dz"), ("x", "z", "dy"), ("y", "z", "dx")
]
for combo in complex_terms:
    name = "*".join(combo)
    term = np.ones_like(x)
    for c in combo:
        term *= features[c]
    features[name] = term

# Trigonometric terms
for var in ["x", "y", "z"]:
    features[f"sin({var})"] = np.sin(df[var])
    features[f"cos({var})"] = np.cos(df[var])

# Combine into DataFrame
feature_df = pd.DataFrame(features)

# DOWNSAMPLE ROWS TO 2000 SAMPLES
feature_df_small = feature_df.iloc[:5000, :].reset_index(drop=True) # the original data from kaggle has 200,000 samples which is too much to compute later on

# SAVE SMALL MATRIX
save_dir = os.path.dirname(file_path)
feature_path_small = os.path.join(save_dir, "rich_feature_matrix_small.csv")
feature_df_small.to_csv(feature_path_small, index=False)

print(f" Downsampled feature matrix (5000 samples) saved at:\n{feature_path_small}")
print(f"Shape: {feature_df_small.shape}")


 Downsampled feature matrix (5000 samples) saved at:
C:\Users\braid\OneDrive\Desktop\lorenze_attractor.csv\rich_feature_matrix_small.csv
Shape: (5000, 66)


Cell 2, Applying the SINDY Algorithm, then comparing the feature library (built in) with the feature library I created

In [151]:
## Cell 2
import pandas as pd
import numpy as np
import itertools
import os
from pysindy import SINDy
from pysindy.feature_library import PolynomialLibrary, IdentityLibrary
from pysindy.optimizers import STLSQ


# File path
file_path = r"C:\Users\braid\OneDrive\Desktop\lorenze_attractor.csv\lorenze_attractor.csv"

# Load Data
df = pd.read_csv(file_path)
df = df.drop(df.columns[0], axis=1)  # drop sample index

expected_cols = ["x", "y", "z", "u", "t"]
for c in expected_cols:
    if c not in df.columns:
        raise ValueError(f"Missing column '{c}' in CSV. Found: {df.columns.tolist()}")

x, y, z, u, t = [df[c].to_numpy() for c in expected_cols]
dt = np.mean(np.diff(t))

# Using Finite Difference for derivatives (FDM)
def finite_diff(signal, dt):
    deriv = np.zeros_like(signal)
    deriv[1:-1] = (signal[2:] - signal[:-2]) / (2 * dt)
    deriv[0] = (signal[1] - signal[0]) / dt
    deriv[-1] = (signal[-1] - signal[-2]) / dt
    return deriv

dx, dy, dz, du = [finite_diff(var, dt) for var in [x, y, z, u]]

# 2. CUSTOM RICH FEATURE MATRIX GENERATION (THETA)

# Building the feature space dictionary
features = {
    "1": np.ones_like(x), # Add constant term for bias
    "x": x, "y": y, "z": z, "u": u, "t": t,
    "dx": dx, "dy": dy, "dz": dz, "du": du
}

vars_basic = ["x", "y", "z", "u"]

# Polynomial terms (up to degree 3)
for var in vars_basic:
    features[f"{var}^2"] = df[var] ** 2
    features[f"{var}^3"] = df[var] ** 3

# Pairwise products (degree 2)
for i, j in itertools.combinations_with_replacement(vars_basic, 2):
    if i != j: # Only cross-terms; x*x is already x^2
        features[f"{i}{j}"] = df[i] * df[j]

# Derivative interactions and higher order terms
deriv_vars = ["dx", "dy", "dz", "du"]
# Derivative interactions (e.g., x*dx)
for var in vars_basic:
    for dvar in deriv_vars:
        features[f"{var}*{dvar}"] = df[var] * features[dvar]

# Cross derivative terms (e.g., dx*dy)
for i, j in itertools.combinations_with_replacement(deriv_vars, 2):
    features[f"{i}*{j}"] = features[i] * features[j]

# Complex higher-order terms (e.g., x*dx*dy)
complex_terms = [
    ("x", "dx", "dy"), ("x", "dy", "dz"), ("y", "dx", "dz"),
    ("z", "dx", "dy"), ("x", "y", "dz"), ("x", "z", "dy"), ("y", "z", "dx")
]
for combo in complex_terms:
    name = "*".join(combo)
    term = np.ones_like(x)
    for c in combo:
        term *= features[c]
    features[name] = term

# Trigonometric terms
for var in ["x", "y", "z"]:
    features[f"sin({var})"] = np.sin(df[var])
    features[f"cos({var})"] = np.cos(df[var])


# Combine into DataFrame and DOWNSAMPLE
feature_df = pd.DataFrame(features)
N_SAMPLES = 5000
feature_df_small = feature_df.iloc[:N_SAMPLES, :].reset_index(drop=True)

# Separate Data for SINDy
deriv_cols = ["dx", "dy", "dz", "du"]

# State Variables (X) - Input for PolynomialLibrary
X = feature_df_small[vars_basic].to_numpy()
# Derivatives (X_dot) - Target for both models
X_dot = feature_df_small[deriv_cols].to_numpy()

# Custom Feature Matrix (Theta) - Input for IdentityLibrary
# Note: Dropping the derivatives and time column
custom_feature_names = [name for name in feature_df_small.columns if name not in deriv_cols and name != "t"]
Theta = feature_df_small[custom_feature_names].to_numpy()


# 3. SINDy MODEL COMPARISON

# A. SINDy with Standard Feature Space (Polynomial Library)
print("\n" + "="*70)
print("A. SINDy with Standard Polynomial Library (Degree 3)")
print("="*70)

poly_library = PolynomialLibrary(degree=3, include_bias=True)

# Initialize the SINDy model
model_poly = SINDy(
    optimizer=STLSQ(threshold=0.05),
    feature_library=poly_library,
)

# Fit the model: Correctly passes feature_names and the time step 't=dt'
model_poly.fit(X, x_dot=X_dot, feature_names=vars_basic, t=dt)

print("\n*** Discovered Equations (Polynomial Library) ***")
model_poly.print(lhs=deriv_cols)


# B. SINDy with Custom Rich Feature Matrix ( My feature library)
print("\n" + "="*70)
print("B. SINDy with Custom Rich Feature Matrix (Your Theta)")
print("="*70)

# Initialize SINDy model (Uses IdentityLibrary for pre-computed Theta)
model_custom = SINDy(
    optimizer=STLSQ(threshold=0.005),
    feature_library=IdentityLibrary(),
)

# Fit the model: Correctly passes feature_names and the time step 't=dt'
model_custom.fit(Theta, x_dot=X_dot, feature_names=custom_feature_names, t=dt)

print("\n*** Discovered Equations (Custom Library) ***")
model_custom.print(lhs=deriv_cols)


# 4. SUMMARY AND COMPARISON ANALYSIS
print("\n" + "="*70)
print("C. Summary of Feature Space Comparison")
print("="*70)

# Print feature sizes (Syntax corrected)
print(f"Polynomial Library Size (Degree 3, 4 vars): {len(model_poly.get_feature_names())}")
print(f"Custom Feature Library Size: {len(model_custom.get_feature_names())}")

# Check model recovery sparsity
print("\nSparsity (Number of active terms):")
print(f"Polynomial Model Active Terms: {model_poly.complexity}")
print(f"Custom Model Active Terms: {model_custom.complexity}")


A. SINDy with Standard Polynomial Library (Degree 3)

*** Discovered Equations (Polynomial Library) ***
dx = -10.000 x + 10.000 y
dy = 0.019 1 + 28.007 x + -1.019 y + -0.002 z + 0.002 x^2 + -0.001 x y + -1.000 x z + 0.025 x u + 0.012 y u + 0.002 z u + 0.092 u^2 + -0.001 x^2 u + 0.001 x y u + -0.019 x u^2 + 0.001 y u^2 + -0.003 z u^2 + -0.035 u^3
dz = 27.020 1 + -6.013 z + 0.321 x^2 + 0.125 x z + 0.234 y^2 + -0.271 y z + 0.112 z^2 + 0.835 z u + -0.110 x^2 u + 0.583 x y u + -0.379 x z u + 4.414 x u^2 + 0.312 y z u + -0.898 y u^2 + -1.268 z u^2 + 10.293 u^3
du = 0.078 z

B. SINDy with Custom Rich Feature Matrix (Your Theta)

*** Discovered Equations (Custom Library) ***
dx = 8.848 y + -38.752 u + 0.505 u^2 + -0.241 xu + 0.051 yu + -85.609 x*du + 81.262 y*du + 19.208 u*du + -8.072 dx*du
dy = 4.050 x + 5.475 y + -0.194 xu + -1.910 x*du + -2.770 y*du + 0.014 u*dy + 0.496 dy*du
dz = -2.667 z + 1.000 x^2 + 0.100 x*dx
du = -0.416 z + 0.250 z*du

C. Summary of Feature Space Comparison
Polynomia

In cell 2, the equations are way too big to be interpreted.

I tried changing the library polynomial degree from 3 to 2. the change is in cell 3 line 26)

In [152]:
## Cell 3
import numpy as np
import matplotlib.pyplot as plt
import pysindy as ps
import pandas as pd

# Load feature space by coping the path of the Excel done in cell 1
feature_path = r"C:\Users\braid\OneDrive\Desktop\lorenze_attractor.csv\rich_feature_matrix_small.csv"
feature_df = pd.read_csv(feature_path)

# Select state variables that I'll predict
state_vars = ['x', 'y', 'z', 'u']
deriv_vars = ['dx', 'dy', 'dz', 'du']

X = feature_df[state_vars].to_numpy()
dXdt = feature_df[deriv_vars].to_numpy()
t = feature_df['t'].to_numpy()
dt = np.mean(np.diff(t))

# Normalize the features
X_mean, X_std = np.mean(X, axis=0), np.std(X, axis=0)
X_norm = (X - X_mean) / X_std

# Defining SINDY
optimizer = ps.STLSQ(threshold=0.1, alpha=0.5)
library = ps.PolynomialLibrary(degree=2)
model = ps.SINDy(optimizer=optimizer, feature_library=library)

model.fit(X_norm, t=dt, x_dot=dXdt, feature_names=state_vars)

# Discovered Equations
print("\n SINDy Model Discovered:\n")
for eq in model.equations():
    print(eq)




 SINDy Model Discovered:

-10.878 1 + -78.511 x + 99.609 y + 0.001 z + -0.007 u + -0.004 x^2 + 0.003 x y + 0.002 x z + -0.014 x u + -0.001 y z + 0.009 y u + -0.003 z u + -0.003 u^2
7.538 1 + 50.323 x + -9.958 y + -10.816 z + 0.008 u + 0.001 x y + -71.183 x z + 0.013 x u + -0.001 y^2 + -0.009 y u + 0.004 z u + 0.004 u^2
-57.471 1 + 0.845 x + 11.891 y + -24.187 z + 0.054 u + 0.026 x^2 + 78.181 x y + -0.007 x z + 0.100 x u + -0.001 y^2 + 0.007 y z + -0.066 y u + -0.004 z^2 + 0.025 z u + 0.025 u^2
1.979 1


The equations from cell 3 are sparse but still i dont think they're interpretable.

So I'll try using different threshold.

In [153]:
## Cell 4
import numpy as np
import matplotlib.pyplot as plt
import pysindy as ps
import pandas as pd

# Load feature space by coping the path of the Excel done in cell 1
feature_path = r"C:\Users\braid\OneDrive\Desktop\lorenze_attractor.csv\rich_feature_matrix_small.csv"
feature_df = pd.read_csv(feature_path)

# Select state variables that I'll predict
state_vars = ['x', 'y', 'z', 'u']
deriv_vars = ['dx', 'dy', 'dz', 'du']

X = feature_df[state_vars].to_numpy()
dXdt = feature_df[deriv_vars].to_numpy()
t = feature_df['t'].to_numpy()
dt = np.mean(np.diff(t))

# Normalize the features
X_mean, X_std = np.mean(X, axis=0), np.std(X, axis=0)
X_norm = (X - X_mean) / X_std

# Defining SINDY
optimizer = ps.STLSQ(threshold=1, alpha=0.5)
library = ps.PolynomialLibrary(degree=2)
model = ps.SINDy(optimizer=optimizer, feature_library=library)

model.fit(X_norm, t=dt, x_dot=dXdt, feature_names=state_vars)

# Discovered Equations
print("\n SINDy Model Discovered:\n")
for eq in model.equations():
    print(eq)




 SINDy Model Discovered:

-10.878 1 + -78.511 x + 99.609 y + 0.001 z + -0.007 u + -0.004 x^2 + 0.003 x y + 0.002 x z + -0.014 x u + -0.001 y z + 0.009 y u + -0.003 z u + -0.003 u^2
7.540 1 + 50.318 x + -9.958 y + -10.814 z + 0.001 u + -0.004 x^2 + 0.003 x y + -71.182 x z + -0.001 y^2 + -0.001 y z + 0.001 z u + 0.001 u^2
-57.471 1 + 0.845 x + 11.891 y + -24.187 z + 0.054 u + 0.026 x^2 + 78.181 x y + -0.007 x z + 0.100 x u + -0.001 y^2 + 0.007 y z + -0.066 y u + -0.004 z^2 + 0.025 z u + 0.025 u^2
1.979 1


From the output of cell 4, the equations discovered looks like they need regularization (too much variables and wide range of coefficients). And maybe try changing the optimizer of the SINDY and use L1 instead of sequentially threshold least square (STLSQ).
In cell 5 I'll try using L1 on the output of sindy.

In [154]:
## Cell 5: SINDy with Lasso (L1) Regularization (Applied to Degree 2 Feature Space)

import numpy as np
from pysindy import SINDy
from pysindy.feature_library import PolynomialLibrary
from sklearn.linear_model import Lasso # Corrected import

# NOTE: This cell requires X, X_dot, dt, vars_basic, and deriv_cols from a previous cell.

# Define the Optimizer
# Using an alpha value (0.005) for moderate sparsity.
LASSO_ALPHA = 0.1

lasso_optimizer = Lasso(
    alpha=LASSO_ALPHA,
    max_iter=100000,
    fit_intercept=False, # Set to False since PolynomialLibrary includes the bias term ('1')
    copy_X=True,
    precompute=True
)

print("\n" + "="*70)
print(f"SINDy with Lasso (L1) Regularization (Alpha={LASSO_ALPHA})")
print("="*70)

#Define the Feature Library (Minimalist Degree 2)
poly_library_deg2 = PolynomialLibrary(degree=2, include_bias=True)

# Initialize and Fit the Model
model_lasso = SINDy(
    optimizer=lasso_optimizer,
    feature_library=poly_library_deg2,
)

# Fit the model (requires X, X_dot, feature_names, and t=dt)
model_lasso.fit(X, x_dot=X_dot, feature_names=vars_basic, t=dt)

# Print Discovered Equations
print("\n*** Discovered Equations (Lasso L1 Optimizer, Alpha=0.005) ***")
model_lasso.print(lhs=deriv_cols)

# Summary (Includes Complexity Calculation Fix)
print("Summary: Lasso L1 Optimizer Results")
print(f"Lasso Alpha (L1 Penalty Strength): {LASSO_ALPHA}")

xi_matrix = model_lasso.optimizer.coef_
complexity_score = np.count_nonzero(xi_matrix)

print(f"Active Terms (Complexity): {complexity_score}")


SINDy with Lasso (L1) Regularization (Alpha=0.1)

*** Discovered Equations (Lasso L1 Optimizer, Alpha=0.005) ***
dx = -2.197 x + 5.419 y + 0.651 z + 0.203 x^2 + -0.009 x y + -0.181 x z + -0.048 y^2 + 0.068 y z + 0.434 y u + -0.034 z^2 + -0.145 z u + 1.275 u^2
dy = 25.863 x + 0.135 y + -0.151 z + -0.048 x^2 + -0.951 x z + 0.015 y^2 + -0.021 y z + 0.009 z^2
dz = -0.001 y + -2.645 z + 0.010 x^2 + 0.991 x y + -0.001 x z + 0.002 y^2 + -0.001 z^2
du = 0.036 y + 0.216 z + 0.012 x^2 + -0.004 x y + 0.001 x z + -0.002 y^2 + -0.002 y z + -0.005 z^2
Summary: Lasso L1 Optimizer Results
Lasso Alpha (L1 Penalty Strength): 0.1
Active Terms (Complexity): 36


The equations in cell 5 resembles the lorenz system but, features with small coefficients should be dropped

In cell 6 I'll try dropping features with small coefficients

I'll compute the average absolute coefficient and drop the coefficients that falls below 5% of that threshold to zero

In [156]:
## Cell 6: Calculate Average Absolute Coefficient Magnitude per Equation

import numpy as np

# Retrieve Coefficients from Lasso Model
# Access the coefficients from the scikit-learn optimizer object
xi_lasso = model_lasso.optimizer.coef_
NUM_STATES = xi_lasso.shape[0] # Number of state variables (rows: dx, dy, dz, du)

print("Average Absolute Coefficient Magnitude per Equation")

# Iterate and Calculate Average

for i in range(NUM_STATES):
    row_coeffs = xi_lasso[i, :]

    # Isolate coefficients that are truly non-zero (above machine precision)
    non_zero_coeffs = row_coeffs[np.abs(row_coeffs) > 1e-12]

    if len(non_zero_coeffs) > 0:
        # Calculate the average absolute magnitude
        avg_abs_magnitude = np.mean(np.abs(non_zero_coeffs))

        # Display the result using the pre-defined derivative names
        print(f"Equation {deriv_cols[i]}: Average Absolute Coefficient Magnitude = {avg_abs_magnitude:.6f}")

    else:
        print(f"Equation {deriv_cols[i]}: No non-zero coefficients found.")


Average Absolute Coefficient Magnitude per Equation
Equation dx: Average Absolute Coefficient Magnitude = 0.888748
Equation dy: Average Absolute Coefficient Magnitude = 3.399129
Equation dz: Average Absolute Coefficient Magnitude = 0.456272
Equation du: Average Absolute Coefficient Magnitude = 0.034794
